In [147]:
import numpy as np
import gzip
import pandas as pd
import pandas_profiling
#!pip install bayesian-optimization
from bayes_opt import BayesianOptimization

In [ ]:
detailed_listing = 'http://data.insideairbnb.com/united-states/ca/los-angeles/2019-12-05/data/listings.csv.gz'
summary_listing = 'http://data.insideairbnb.com/united-states/ca/los-angeles/2019-12-05/visualisations/listings.csv'

In [2]:
det_list = pd.read_csv(detailed_listing, compression='gzip', error_bad_lines=False, warn_bad_lines=False)

print(det_list.shape)
det_list.head()

(40125, 106)


C:\Users\seanh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (43,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,109,https://www.airbnb.com/rooms/109,20191205175409,2019-12-06,Amazing bright elegant condo park front *UPGRA...,"*** Unit upgraded with new bamboo flooring, br...","*** Unit upgraded with new bamboo flooring, br...","*** Unit upgraded with new bamboo flooring, br...",none,NaN,...,f,f,strict_14_with_grace_period,t,f,1,1,0,0,0.02
1,344,https://www.airbnb.com/rooms/344,20191205175409,2019-12-06,Family perfect;Pool;Near Studios!,This home is perfect for families; aspiring ch...,"Cheerful & comfortable; near studios, amusemen...",This home is perfect for families; aspiring ch...,none,Quiet-yet-close to all the fun in LA! Hollywoo...,...,t,f,flexible,f,f,1,1,0,0,0.19
2,2708,https://www.airbnb.com/rooms/2708,20191205175409,2019-12-06,Mirrored Mini-Suite with Fireplace - W. Hollywood,Our best memory foam pillows you'll ever sleep...,Flickering fireplace. Blendtec® Designer 625 ...,Our best memory foam pillows you'll ever sleep...,none,We are minutes away from the Mentor Language I...,...,t,f,strict_14_with_grace_period,f,f,2,0,2,0,0.33
3,2732,https://www.airbnb.com/rooms/2732,20191205175409,2019-12-06,Zen Life at the Beach,NaN,This is a three story townhouse with the follo...,This is a three story townhouse with the follo...,none,NaN,...,f,f,strict_14_with_grace_period,f,f,2,1,1,0,0.19
4,2864,https://www.airbnb.com/rooms/2864,20191205175409,2019-12-07,*Upscale Professional Home with Beautiful Studio*,Centrally located.... Furnished with 42 inch S...,The space is furnished with Thomasville furnit...,Centrally located.... Furnished with 42 inch S...,none,What makes the neighborhood unique is that the...,...,f,f,strict_14_with_grace_period,f,f,1,1,0,0,NaN


In [3]:
summary_list = pd.read_csv(summary_listing, error_bad_lines=False, warn_bad_lines=False )

print(summary_list.shape)
summary_list.head()

(40125, 16)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,109,Amazing bright elegant condo park front *UPGRA...,521,Paolo,Other Cities,Culver City,33.98209,-118.38494,Entire home/apt,122,30,2,2016-05-15,0.02,1,248
1,344,Family perfect;Pool;Near Studios!,767,Melissa,Other Cities,Burbank,34.16562,-118.33458,Entire home/apt,168,2,8,2019-10-19,0.19,1,62
2,2708,Mirrored Mini-Suite with Fireplace - W. Hollywood,3008,Chas.,City of Los Angeles,Hollywood,34.09768,-118.34602,Private room,79,30,22,2019-08-31,0.33,2,252
3,2732,Zen Life at the Beach,3041,Yoga Priestess,Other Cities,Santa Monica,34.00475,-118.48127,Private room,140,1,20,2019-10-03,0.19,2,344
4,2864,*Upscale Professional Home with Beautiful Studio*,3207,Bernadine,Other Cities,Bellflower,33.87619,-118.11397,Entire home/apt,80,2,0,NaN,NaN,1,0


In [4]:
calendar_url = 'http://data.insideairbnb.com/united-states/ca/los-angeles/2019-12-05/data/calendar.csv.gz'

In [116]:
calendar_df = pd.read_csv(calendar_url, compression='gzip', warn_bad_lines=False, error_bad_lines=False)

print(calendar_df.shape)
calendar_df.head()

(14670849, 7)


,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,46858,2019-12-06,f,$300.00,$300.00,3.0,30.0
1,130915,2019-12-06,f,$90.00,$90.00,1.0,730.0
2,130915,2019-12-07,f,$90.00,$90.00,1.0,730.0
3,130915,2019-12-08,f,$90.00,$90.00,1.0,730.0
4,130915,2019-12-09,f,$90.00,$90.00,1.0,730.0


In [117]:
calendar_df['date'] = pd.to_datetime(calendar_df['date'])
calendar_df['year'] = calendar_df['date'].dt.year
calendar_df['month'] = calendar_df['date'].dt.month
calendar_df['day'] = calendar_df['date'].dt.day
calendar_df = calendar_df.drop(columns='date')
calendar_df['available'] = calendar_df['available'].astype(str)
calendar_df['price'] = calendar_df['price'].replace('$','')
calendar_df['adjusted_price'] = calendar_df['adjusted_price'].str[1:]
# Take care of NAs so this code can run
calendar_df['minimum_nights'].fillna(calendar_df['minimum_nights'].mode()[0], inplace=True)
calendar_df['minimum_nights'] = calendar_df['minimum_nights'].astype(int)
calendar_df['maximum_nights'].fillna(calendar_df['maximum_nights'].mode()[0], inplace=True)
calendar_df['maximum_nights'] = calendar_df['maximum_nights'].astype(int)
calendar_df['price'] = calendar_df['price'].str[1:]
calendar_df.head()

,listing_id,available,price,adjusted_price,minimum_nights,maximum_nights,year,month,day
0,46858,f,300.00,300.00,3,30,2019,12,6
1,130915,f,90.00,90.00,1,730,2019,12,6
2,130915,f,90.00,90.00,1,730,2019,12,7
3,130915,f,90.00,90.00,1,730,2019,12,8
4,130915,f,90.00,90.00,1,730,2019,12,9


In [118]:
list(summary_list.columns)

['id',
 'name',
 'host_id',
 'host_name',
 'neighbourhood_group',
 'neighbourhood',
 'latitude',
 'longitude',
 'room_type',
 'price',
 'minimum_nights',
 'number_of_reviews',
 'last_review',
 'reviews_per_month',
 'calculated_host_listings_count',
 'availability_365']

In [119]:
list(det_list.columns)

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'neighborhood_overview',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'latitude',
 'longitude',
 'is_location_exact',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'amenities',


In [120]:
#pandas_profiling.ProfileReport(det_list)

In [121]:
det_list.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,109,https://www.airbnb.com/rooms/109,20191205175409,2019-12-06,Amazing bright elegant condo park front *UPGRA...,"*** Unit upgraded with new bamboo flooring, br...","*** Unit upgraded with new bamboo flooring, br...","*** Unit upgraded with new bamboo flooring, br...",none,NaN,...,f,f,strict_14_with_grace_period,t,f,1,1,0,0,0.02
1,344,https://www.airbnb.com/rooms/344,20191205175409,2019-12-06,Family perfect;Pool;Near Studios!,This home is perfect for families; aspiring ch...,"Cheerful & comfortable; near studios, amusemen...",This home is perfect for families; aspiring ch...,none,Quiet-yet-close to all the fun in LA! Hollywoo...,...,t,f,flexible,f,f,1,1,0,0,0.19
2,2708,https://www.airbnb.com/rooms/2708,20191205175409,2019-12-06,Mirrored Mini-Suite with Fireplace - W. Hollywood,Our best memory foam pillows you'll ever sleep...,Flickering fireplace. Blendtec® Designer 625 ...,Our best memory foam pillows you'll ever sleep...,none,We are minutes away from the Mentor Language I...,...,t,f,strict_14_with_grace_period,f,f,2,0,2,0,0.33
3,2732,https://www.airbnb.com/rooms/2732,20191205175409,2019-12-06,Zen Life at the Beach,NaN,This is a three story townhouse with the follo...,This is a three story townhouse with the follo...,none,NaN,...,f,f,strict_14_with_grace_period,f,f,2,1,1,0,0.19
4,2864,https://www.airbnb.com/rooms/2864,20191205175409,2019-12-07,*Upscale Professional Home with Beautiful Studio*,Centrally located.... Furnished with 42 inch S...,The space is furnished with Thomasville furnit...,Centrally located.... Furnished with 42 inch S...,none,What makes the neighborhood unique is that the...,...,f,f,strict_14_with_grace_period,f,f,1,1,0,0,NaN


In [122]:
columns = ['price', 'beds', 'bedrooms', 'bathrooms', 'zipcode', 'neighbourhood', 
           'property_type', 'room_type', 'accommodates', 'guests_included',
           'minimum_nights', 'instant_bookable']

train = det_list[columns]

train.head()

,price,beds,bedrooms,bathrooms,zipcode,neighbourhood,property_type,room_type,accommodates,guests_included,minimum_nights,instant_bookable
0,$122.00,3.0,2.0,2.0,90230,Culver City,Condominium,Entire home/apt,6,3,30,f
1,$168.00,3.0,3.0,1.0,91505,Burbank,House,Entire home/apt,6,6,2,t
2,$79.00,1.0,1.0,1.5,90046,Hollywood,Apartment,Private room,1,1,30,t
3,$140.00,1.0,1.0,1.0,90405,Santa Monica,Apartment,Private room,1,1,1,f
4,$80.00,1.0,1.0,1.0,90706,Bellflower,Apartment,Entire home/apt,2,1,2,f


In [123]:
train['price'] = train['price'].str.replace('$','')
train['price'] = train['price'].str.replace(',','').astype('float')

train['beds'] = train['beds'].fillna(train['beds'].mode()[0])
train['beds'] = train['beds'].astype(int)

train['zipcode'] = train['zipcode'].str.strip()
train['zipcode'] = np.where(train['zipcode'].str.contains('ca', case = False), '', train['zipcode'])
train['zipcode'] = train['zipcode'].str.replace(' ', '')
train['zipcode'] = train['zipcode'].str.replace('139SValenciaAve,Glendora.', '')
train['zipcode'] = train['zipcode'].str[:5]
train['zipcode'] = train['zipcode'].replace('',np.nan)
train['zipcode'] = train['zipcode'].astype('float')

train['instant_bookable'] = np.where(train['instant_bookable'].str.contains('t'), 1, 0)


C:\Users\seanh\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\seanh\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\seanh\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [124]:
train.head()

,price,beds,bedrooms,bathrooms,zipcode,neighbourhood,property_type,room_type,accommodates,guests_included,minimum_nights,instant_bookable
0,122.0,3,2.0,2.0,90230.0,Culver City,Condominium,Entire home/apt,6,3,30,0
1,168.0,3,3.0,1.0,91505.0,Burbank,House,Entire home/apt,6,6,2,1
2,79.0,1,1.0,1.5,90046.0,Hollywood,Apartment,Private room,1,1,30,1
3,140.0,1,1.0,1.0,90405.0,Santa Monica,Apartment,Private room,1,1,1,0
4,80.0,1,1.0,1.0,90706.0,Bellflower,Apartment,Entire home/apt,2,1,2,0


In [125]:
train.dtypes

price               float64
beds                  int32
bedrooms            float64
bathrooms           float64
zipcode             float64
neighbourhood        object
property_type        object
room_type            object
accommodates          int64
guests_included       int64
minimum_nights        int64
instant_bookable      int32
dtype: object

In [126]:
import xgboost as xgb
from sklearn.compose import ColumnTransformer
import category_encoders as ce

In [127]:
columns = ['price', 'beds', 'bedrooms', 'bathrooms', 'zipcode', 'neighbourhood', 
           'property_type', 'room_type', 'accommodates', 'guests_included',
           'minimum_nights', 'instant_bookable']

train = det_list[columns]

train.head()

,price,beds,bedrooms,bathrooms,zipcode,neighbourhood,property_type,room_type,accommodates,guests_included,minimum_nights,instant_bookable
0,$122.00,3.0,2.0,2.0,90230,Culver City,Condominium,Entire home/apt,6,3,30,f
1,$168.00,3.0,3.0,1.0,91505,Burbank,House,Entire home/apt,6,6,2,t
2,$79.00,1.0,1.0,1.5,90046,Hollywood,Apartment,Private room,1,1,30,t
3,$140.00,1.0,1.0,1.0,90405,Santa Monica,Apartment,Private room,1,1,1,f
4,$80.00,1.0,1.0,1.0,90706,Bellflower,Apartment,Entire home/apt,2,1,2,f


In [128]:
train['price'] = train['price'].str.replace('$','')
train['price'] = train['price'].str.replace(',','').astype('float')

train['beds'] = train['beds'].fillna(train['beds'].mode()[0])
#train['beds'].fillna(train['beds'].mode()[0], inplace=True)
train['beds'] = train['beds'].astype(int)

train['zipcode'] = train['zipcode'].str.strip()
train['zipcode'] = np.where(train['zipcode'].str.contains('ca', case = False), '', train['zipcode'])
train['zipcode'] = train['zipcode'].str.replace(' ', '')
train['zipcode'] = train['zipcode'].str.replace('139SValenciaAve,Glendora.', '')
train['zipcode'] = train['zipcode'].str[:5]
train['zipcode'] = train['zipcode'].replace('',np.nan)
train['zipcode'] = train['zipcode'].astype(str)

train['instant_bookable'] = np.where(train['instant_bookable'].str.contains('t'), 1, 0)

C:\Users\seanh\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\seanh\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\seanh\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [129]:
train.dtypes

price               float64
beds                  int32
bedrooms            float64
bathrooms           float64
zipcode              object
neighbourhood        object
property_type        object
room_type            object
accommodates          int64
guests_included       int64
minimum_nights        int64
instant_bookable      int32
dtype: object

In [130]:
##next will do a linear regression model for a baseline

In [131]:
# Features we want
categorical_features = ['zipcode','neighbourhood','property_type','room_type']
numeric_features = ['price', 'beds', 'bedrooms', 'bathrooms','accommodates','guests_included','minimum_nights','instant_bookable']

In [132]:
#categorical_counts = train.select_dtypes(exclude='number').nunique().sort_values()
categorical_counts = []
categorical_counts.append(train[categorical_features].nunique().sort_values())
high_cardi_cols = []
low_cardi_cols = []
for i in categorical_features:
    if train[i].nunique() > 150:
        high_cardi_cols.append(i)
    else:
        low_cardi_cols.append(i)

In [133]:
print(categorical_counts, high_cardi_cols, low_cardi_cols)

[room_type          4
property_type     44
neighbourhood    165
zipcode          295
dtype: int64] ['zipcode', 'neighbourhood'] ['property_type', 'room_type']


In [134]:
ct = ColumnTransformer([("oe", ce.OrdinalEncoder(), high_cardi_cols),
                        ("ohe", ce.OneHotEncoder(use_cat_names=True), low_cardi_cols)])

In [136]:
# We'll impute missing values using the median for numeric columns and the most
# common value for string columns.

from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)

train = DataFrameImputer().fit_transform(train)

In [137]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
for feature in categorical_features:
    train[feature] = label_encoder.fit_transform(train[feature])

In [138]:
train.head()

,price,beds,bedrooms,bathrooms,zipcode,neighbourhood,property_type,room_type,accommodates,guests_included,minimum_nights,instant_bookable
0,122.0,3,2.0,2.0,77,29,15,0,6,3,30,0
1,168.0,3,3.0,1.0,229,19,25,0,6,6,2,1
2,79.0,1,1.0,1.5,50,59,1,2,1,1,30,1
3,140.0,1,1.0,1.0,113,118,1,2,1,1,1,0
4,80.0,1,1.0,1.0,133,14,1,0,2,1,2,0


In [139]:
features = numeric_features + categorical_features
target = 'price'
train_features = train[features]

In [140]:
train_labels = train_features['price']
train_features = train_features.drop(columns=['price'])

In [141]:
X_train = train_features
y_train = train_labels

X_train.shape, y_train.shape

((40125, 11), (40125,))

In [142]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size = 0.80, 
                                                  test_size = 0.20, random_state=42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((32100, 11), (8025, 11), (32100,), (8025,))

In [168]:
from xgboost import XGBRegressor

eval_set = [(X_train, y_train), 
            (X_val, y_val)]

model = XGBRegressor(n_estimators=1000, n_jobs=-1)
model.fit(X_train, y_train, eval_set=eval_set, eval_metric='rmse', 
          early_stopping_rounds=100)

[00:31:19] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:658.312	validation_1-rmse:707.754
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[1]	validation_0-rmse:635.297	validation_1-rmse:688.316
[2]	validation_0-rmse:615.692	validation_1-rmse:672.534
[3]	validation_0-rmse:597.619	validation_1-rmse:658.274
[4]	validation_0-rmse:581.556	validation_1-rmse:641.798
[5]	validation_0-rmse:567.944	validation_1-rmse:627.606
[6]	validation_0-rmse:556.12	validation_1-rmse:615.794
[7]	validation_0-rmse:546.558	validation_1-rmse:608.996
[8]	validation_0-rmse:537.855	validation_1-rmse:600.314
[9]	validation_0-rmse:530.433	validation_1-rmse:592.476
[10]	validation_0-rmse:523.564	validation_1-rmse:585.213
[11]	validation_0-rmse:517.989	validation_1-rmse:579.602
[12

[138]	validation_0-rmse:428.502	validation_1-rmse:514.57
[139]	validation_0-rmse:428.223	validation_1-rmse:514.022
[140]	validation_0-rmse:428.182	validation_1-rmse:513.998
[141]	validation_0-rmse:427.998	validation_1-rmse:514.014
[142]	validation_0-rmse:427.856	validation_1-rmse:513.994
[143]	validation_0-rmse:427.781	validation_1-rmse:513.916
[144]	validation_0-rmse:426.985	validation_1-rmse:513.67
[145]	validation_0-rmse:426.663	validation_1-rmse:513.348
[146]	validation_0-rmse:426.414	validation_1-rmse:513.359
[147]	validation_0-rmse:426.102	validation_1-rmse:513.227
[148]	validation_0-rmse:425.913	validation_1-rmse:513.122
[149]	validation_0-rmse:425.855	validation_1-rmse:513.122
[150]	validation_0-rmse:425.762	validation_1-rmse:513.075
[151]	validation_0-rmse:425.509	validation_1-rmse:513.002
[152]	validation_0-rmse:425.456	validation_1-rmse:512.901
[153]	validation_0-rmse:425.383	validation_1-rmse:512.903
[154]	validation_0-rmse:425.316	validation_1-rmse:512.886
[155]	validation

[280]	validation_0-rmse:409.871	validation_1-rmse:506.923
[281]	validation_0-rmse:409.857	validation_1-rmse:506.915
[282]	validation_0-rmse:409.554	validation_1-rmse:506.712
[283]	validation_0-rmse:409.171	validation_1-rmse:506.677
[284]	validation_0-rmse:408.86	validation_1-rmse:506.674
[285]	validation_0-rmse:408.736	validation_1-rmse:506.553
[286]	validation_0-rmse:408.608	validation_1-rmse:506.592
[287]	validation_0-rmse:408.596	validation_1-rmse:506.578
[288]	validation_0-rmse:408.58	validation_1-rmse:506.575
[289]	validation_0-rmse:408.492	validation_1-rmse:506.617
[290]	validation_0-rmse:408.478	validation_1-rmse:506.583
[291]	validation_0-rmse:408.332	validation_1-rmse:506.849
[292]	validation_0-rmse:407.925	validation_1-rmse:506.848
[293]	validation_0-rmse:407.879	validation_1-rmse:506.776
[294]	validation_0-rmse:407.794	validation_1-rmse:506.765
[295]	validation_0-rmse:407.761	validation_1-rmse:506.73
[296]	validation_0-rmse:407.622	validation_1-rmse:506.85
[297]	validation_0

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=-1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [169]:
# Convert processed data into an xgb.DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train)

In [170]:
def xgb_evaluate(colsample_bytree, gamma, learning_rate, max_delta_step, max_depth_step, min_child_weight, n_estimators, subsample):
    params = {'colsample_bytree': colsample_bytree,
              'eta': 0.1,
              'eval_metric': 'rmse',
              'gamma': gamma,
              'learning_rate': learning_rate,
              'max_delta_step': max_delta_step,
              'max_depth_step': int(max_depth_step),
              'min_child_weight': min_child_weight,
              'n_estimators': n_estimators,
              'seed': 9,
              'subsample': subsample}
    # Use around 1000 boosting rounds in the full model
    cv_result = xgb.cv(params, dtrain, num_boost_round=1000, nfold=3)    
    
    # Bayesian optimization only knows how to maximize, not minimize, so return the negative RMSE
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

In [172]:
xgb_bo = BayesianOptimization(xgb_evaluate, {'colsample_bytree': (0.3, 1),
                                             'gamma': (0, 5),
                                             'learning_rate':(0.01,0.5),
                                             'max_delta_step': (0, 10),
                                             'max_depth_step': (1, 6),
                                             'min_child_weight':(0,5),
                                             'n_estimators':(850, 1200),
                                             'subsample': (0.4, 1.0)})
# Use the expected improvement acquisition function to handle negative numbers
# Optimally needs quite a few more initiation points and number of iterations
xgb_bo.maximize(init_points=6, n_iter=10, acq='ei')

|   iter    |  target   | colsam... |   gamma   | learni... | max_de... | max_de... | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------------------------------
|  1        | -638.7    |  0.4565   |  1.791    |  0.06279  |  3.463    |  4.017    |  2.173    |  852.3    |  0.9904   |
|  2        | -633.6    |  0.5978   |  1.21     |  0.3467   |  0.7492   |  5.619    |  3.846    |  1.172e+0 |  0.5928   |
|  3        | -583.5    |  0.9893   |  0.9412   |  0.4492   |  1.834    |  2.527    |  2.484    |  1.183e+0 |  0.7534   |
|  4        | -599.9    |  0.7329   |  2.178    |  0.1828   |  3.257    |  1.758    |  0.2443   |  941.0    |  0.9898   |
|  5        | -489.3    |  0.6085   |  1.769    |  0.4861   |  9.159    |  4.283    |  4.046    |  862.0    |  0.7882   |
|  6        | -610.3    |  0.5617   |  0.02468  |  0.1129   |  4.265    |  3.352    |  4.875    |  1.141e+0 |  0.8228   |
|  7        | -524.1    

|  41       | -530.8    |  0.7365   |  2.285    |  0.2082   |  9.556    |  3.934    |  3.993    |  861.8    |  0.6142   |
|  42       | -572.1    |  0.6169   |  4.853    |  0.2152   |  4.788    |  5.562    |  4.543    |  975.0    |  0.5237   |
|  43       | -575.3    |  0.8226   |  1.281    |  0.1117   |  8.687    |  3.869    |  4.121    |  861.5    |  0.4283   |
|  44       | -495.0    |  0.3481   |  1.667    |  0.4614   |  9.256    |  4.106    |  4.293    |  862.0    |  0.6673   |
|  45       | -489.0    |  0.7647   |  1.97     |  0.483    |  9.391    |  3.406    |  4.242    |  861.6    |  0.8635   |
|  46       | -550.2    |  0.8519   |  1.928    |  0.1553   |  9.258    |  3.097    |  3.99     |  861.5    |  0.7198   |
|  47       | -511.7    |  0.5611   |  2.051    |  0.2799   |  9.681    |  3.915    |  4.998    |  861.9    |  0.7593   |
|  48       | -568.8    |  0.5248   |  4.659    |  0.2226   |  4.802    |  5.36     |  0.7612   |  880.4    |  0.9334   |
|  49       | -489.9    

|  82       | -558.4    |  0.908    |  3.455    |  0.1859   |  6.753    |  3.776    |  0.882    |  925.7    |  0.7156   |
|  83       | -498.0    |  0.5943   |  2.711    |  0.4938   |  7.134    |  4.533    |  1.481    |  1.179e+0 |  0.894    |
|  84       | -643.5    |  0.8939   |  2.704    |  0.04576  |  4.014    |  4.241    |  0.9372   |  1.073e+0 |  0.5635   |
|  85       | -560.3    |  0.5747   |  1.346    |  0.2618   |  4.733    |  3.862    |  2.122    |  1.184e+0 |  0.671    |
|  86       | -652.9    |  0.4807   |  3.129    |  0.02922  |  4.277    |  4.493    |  4.0      |  1.06e+03 |  0.9995   |
|  87       | -496.6    |  0.4141   |  1.79     |  0.4036   |  9.271    |  3.355    |  4.794    |  862.2    |  0.8951   |
|  88       | -503.7    |  0.8523   |  2.109    |  0.3391   |  9.585    |  5.197    |  2.363    |  958.0    |  0.5558   |
|  89       | -490.4    |  0.5777   |  0.3552   |  0.4611   |  9.499    |  3.075    |  2.589    |  914.6    |  0.8605   |
|  90       | -520.5    

In [191]:
# Write our ensemble method - train 11 models, and weight them all equally by finding the mode of the votes for each well.
# The majority vote for each well is the consensus, which will be our predictions
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import random

random.seed(42)
count = 0
score = 0
scores = 0
prediction_votes=pd.DataFrame()
for num in range(1,2):
    seed = random.randint(1,9000)
    model = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1, 
                          colsample_bynode=1, colsample_bytree=1, gamma=0,
                          importance_type='gain', learning_rate=0.1, max_delta_step=0,
                          max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
                          n_jobs=-1, nthread=None, objective='reg:squarederror', random_state=0,
                          reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
                          silent=None, subsample=1, verbosity=1).fit(X_train, y_train)
    predictions = model.predict(X_val)
    #score = accuracy_score(y_val, predictions)
    prediction_votes[num] = predictions
    scores += score
    count += 1
    print(score)
    y_pred_proba = model.predict_proba(X_train)[:,1]
    print('Test ROC AUC:', roc_auc_score(y_train, y_pred_proba))

print(f"The average accuracy score using these hyperparams over {count} iterations is {scores/count}")

0


AttributeError: 'XGBRegressor' object has no attribute 'predict_proba'